In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks



In [2]:
# q: what's a callback?
# a: a function that is passed as an argument to another function and is called by that function
# example: a function that is called when a button is pressed

### Callbacks as GUI events

In [3]:
import ipywidgets as widgets

From the [ipywidget docs](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Events.html):

- *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

In [4]:
w = widgets.Button(description='Click me')

In [5]:
w

Button(description='Click me', style=ButtonStyle())

In [6]:
def f(o): print('hi')

In [7]:
w.on_click(f)

*NB: When callbacks are used in this way they are often called "events".*

### Creating your own callback

In [8]:
from time import sleep

In [9]:
# Define a function called slow_calculation
def slow_calculation():
    # Initialise a variable called res
    res = 0
    # Iterate over values from 0 to 4
    for i in range(5):
        # Add the square of the current value of i to res
        res += i*i
        # Wait for 1 second
        sleep(1)
    # Return the value of res
    return res

In [10]:
slow_calculation()

30

In [11]:
def slow_calculation(cb=None):
    # We start with a result of zero.
    res = 0
    # For each integer between 0 and 4 inclusive,
    for i in range(5):
        # We add the square of the integer to the result.
        res += i*i
        # Then wait for 1 second,
        sleep(1)
        # And if we have been given a callback function,
        if cb: cb(i)
    # We return the result.
    return res

In [12]:
def show_progress(epoch): print(f"Awesome! We've finished epoch {epoch}!")

In [13]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

### Lambdas and partials

In [14]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [15]:
# Define a function
def show_progress(exclamation, epoch):
    # Print the message
    print(f"{exclamation}! We've finished epoch {epoch}!")

In [16]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [17]:
def make_show_progress(exclamation):
    # We are defining a function that returns a function. The inner function
    # is the function that is returned.
    def _inner(epoch):
        # We are defining a function that prints a string. The string
        # is computed by substituting the parameter `epoch` into the
        # formatting string.
        print(f"{exclamation}! We've finished epoch {epoch}!")
    # We are returning the inner function.
    return _inner

In [18]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

In [19]:
from functools import partial

In [20]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [21]:
f2 = partial(show_progress, "OK I guess")

### Callbacks as callable classes

In [22]:
class ProgressShowingCallback():
    # Define an __init__ method that takes an exclamation mark as input
    def __init__(self, exclamation="Awesome"): self.exclamation = exclamation

    # Define a __call__ method that takes an epoch number as input and prints a message 
    def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [23]:
cb = ProgressShowingCallback("Just super")

In [24]:

slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

### Multiple callback funcs; `*args` and `**kwargs`

In [25]:
def f(*a, **b): print(f"args: {a}; kwargs: {b}")

In [26]:
f(3, 'a', thing1="hello")

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [27]:
def g(a,b,c=0): print(a,b,c)

In [28]:
args = [1,2]
kwargs = {'c':3}
g(*args, **kwargs)

1 2 3


In [29]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

In [30]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc (self, *args, **kwargs): print(f"Done step")

In [31]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [32]:
class PrintStatusCallback():
    # callback that prints the epoch number before and after the calculation of the square of the epoch number
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}") # print the epoch number before the calculation
    def after_calc (self, epoch, val, **kwargs): print(f"After {epoch}: {val}") # print the epoch number and the value of the square of the epoch number after the calculation

In [33]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

### Modifying behavior

In [34]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb,'before_calc'): cb.before_calc(i) # check if the callback has a before_calc method and if so, call it
        res += i*i
        sleep(1)
        if cb and hasattr(cb,'after_calc'): # check if the callback has an after_calc method and if so, call it
            if cb.after_calc(i, res): # check if the callback wants to cancel the calculation and if so, break
                print("stopping early") # print a message
                break
    return res

In [35]:
class PrintAfterCallback():
    def after_calc (self, epoch, val):
        print(f"After {epoch}: {val}")
        if val>10: return True

In [36]:
slow_calculation(PrintAfterCallback())

After 0: 0
After 1: 1
After 2: 5
After 3: 14
stopping early


14

In [37]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res = cb,0 # slow calculator has a callback and a result
    
    def callback(self, cb_name, *args): # callback function takes a callback name and any number of arguments
        if not self.cb: return
        cb = getattr(self.cb,cb_name, None) # get the callback function from the caller
        if cb: return cb(self, *args) # if the callback exists, call it with the slow calculator and the arguments

    def calc(self): # calc function
        for i in range(5): # iterate 5 times
            self.callback('before_calc', i) # call the callback with the name "before_calc"
            self.res += i*i # add the square of i to the result
            sleep(1) # wait 1 second
            if self.callback('after_calc', i): # call the callback with the name "after_calc"
                print("stopping early") # if the callback returns True, stop early
                break

In [38]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [39]:
calculator = SlowCalculator(ModifyingCallback())

In [40]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15

## `__dunder__` thingies

Anything that looks like `__this__` is, in some way, *special*. Python, or some library, can define some functions that they will call at certain documented times. For instance, when your class is setting up a new object, python will call `__init__`. These are defined as part of the python [data model](https://docs.python.org/3/reference/datamodel.html#object.__init__).

For instance, if python sees `+`, then it will call the special method `__add__`. If you try to display an object in Jupyter (or lots of other places in Python) it will call `__repr__`.

In [41]:
# q: what's a python data model in plain language?
# a: it's a way of representing data in a way that's easy to manipulate and use

In [42]:
class SloppyAdder():
    def __init__(self,o): self.o=o # sloppy adder has an object o
    def __add__(self,b): return SloppyAdder(self.o + b.o + 0.01) # sloppy adder adds the object o of the sloppy adder to the object o of the other sloppy adder and adds 0.01
    def __repr__(self): return str(self.o) # sloppy adder returns the object o as a string

In [43]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

In [44]:
class SloppyRandomizer():
    import numpy as np
    from typing import Any
    def __init__(self,mu:float,sigma:float): self.mu,self.sigma = mu,sigma
    def __repr__(self) -> str: return str(self.np.random.normal(self.mu,self.sigma))
    def multiply(self,o:Any) -> Any: return self.np.random.normal(self.mu,self.sigma) * o
    def __call__(self) -> float: return self.np.random.normal(self.mu,self.sigma)

In [45]:
a = SloppyRandomizer(0,1)
a

# multiply the random number by 5
a.multiply(a.sigma)

0.32166729098083763

In [46]:
type(a())

float

Special methods you should probably know about (see data model link above) are:

- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`

### `__getattr__` and `getattr`

In [47]:
# q: what is __getattr__ or getattr do?
# a: it's a way of getting an attribute of an object

In [48]:
class A: a,b=1,2

In [49]:
a = A() # create an object a of class A

In [50]:
a.b # get the attribute b of object a

2

In [51]:
getattr(a, 'b') # get the attribute b of object a

2

In [52]:
# get the attribute b or a of object a depending on the result of a random number
getattr(a, 'b' if random.random()>0.5 else 'a') 

1

In [53]:
class B:
    a,b=1,2 # class B has attributes a and b
    def __getattr__(self, k): 
        if k[0]=='_': raise AttributeError(k) # if the first character of the attribute name is an underscore, raise an attribute error 
        return f'Hello from {k}' # otherwise, return a string

In [54]:
b = B()

In [55]:
b.a

1

In [56]:
b.foo

'Hello from foo'

In [57]:
# __getitem__ is a way of getting an item from an object
# make an torch array and show how to use getitem for it
a = torch.arange(9).reshape(3,3)

# get items from the first row with __getitem__
a[0]

tensor([0, 1, 2])

In [89]:
from typing import Any


class array_operations:
    def __init__(self, numbers): self.numbers = numbers # array operations has an array of numbers
    def __getitem__(self, idx): return self.numbers[idx] # get the item at index idx of the array arr
    def __len__(self): return len(self.numbers) # get the length of the array arr
    def __repr__(self): return str(self.numbers) # return the array arr as a string
    def __del__ (self): print("deleted arr") # print a message when the array is deleted

In [90]:
# create an array operations object
arr = array_operations(torch.arange(9).reshape(3,3))

# get the whole array
getattr(arr, 'numbers')

deleted


tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [91]:
# get the first row
arr[0]

tensor([0, 1, 2])

In [92]:
# multiply the array by 2
arr[0]*2

tensor([0, 2, 4])

In [93]:
# get the first row
print (arr[0])

tensor([0, 1, 2])


In [94]:
# get the length of the array
arr.__len__()

3

In [95]:
# get the array as a string
arr.__repr__()

'tensor([[0, 1, 2],\n        [3, 4, 5],\n        [6, 7, 8]])'

In [96]:
# q: what is __setattr__ or setattr do?
# a: it's a way of setting an attribute of an object

# use setattr in array_operations to set an attribute
setattr(arr, 'numbers', torch.arange(9).reshape(3,3))

# get the attribute
getattr(arr, 'numbers')

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])

In [97]:
# q: what is __delattr__ or delattr do?
# a: it's a way of deleting an attribute of an object

# delete the attribute
arr.__del__()

deleted arr


In [116]:
# q: what is __call__ or call do?
# a: it's a way of calling an object as a function

# create a class that can be called as a function
# adder can take any number of arguments
class adder:
    def __call__(self, *args): return sum(args) # return the sum of the arguments

# create an adder object
add = adder()

# call the adder object with 1,2,3
add(1,2,3)

6

In [112]:
# q: what is __new__ or new do?
# a: it's a way of creating an object

import math

# use class inheritance to create a new object
class multiply(adder):
    def __new__(cls, *args): return math.prod(args)

# create a multiply object
multiply(1,2,3)

6

In [3]:
# q: what is __enter__ or enter do?
# a: it's a way of entering a context

# create a class that can be used in a context
class Context():
    def __enter__(self): print("entering context") # print a message when entering the context
    def __exit__(self, exc_type, exc_value, traceback): print("exiting context") # print a message when exiting the context

# create a context object
c = Context()

# enter the context
with c: 
    # read in gitignore
    with open(".gitignore", "r") as f:
        # print the first line
        print(f.readline())
    # exit the context
    print("exited context")
    


entering context
# Byte-compiled / optimized / DLL files

exited context
exiting context


In [ ]:
# summary of notebook
# reviewed callbacks: a way of passing a function to another function
# reviewed python data model: a way of representing data in a way that's easy to manipulate and use
# reviewed getattr: a way of getting an attribute of an object
# reviewed setattr: a way of setting an attribute of an object
# reviewed delattr: a way of deleting an attribute of an object
# reviewed call: a way of calling an object as a function
# reviewed new: a way of creating an object
# reviewed enter: a way of entering a context
# reviewed exit: a way of exiting a context
